In [2]:
"""
Created By Kevin Du for OSGC Scholarship Program. Credits to "Neural network powered COVID-19 spread forecasting model"\
by Michał Wieczorek, Jakub Siłka, Marcin Woźniak for architecture and data processing.

https://doi.org/10.1016/j.chaos.2020.110203

~~notes notes notes
Purpose:
Intended result:

"""

'\nCreated By Kevin Du for OSGC Scholarship Program. Credits to "Neural network powered COVID-19 spread forecasting model"by Michał Wieczorek, Jakub Siłka, Marcin Woźniak for architecture and data processing.\n\n~~notes notes notes\n\n\n'

In [3]:
import tensorflow as tf
from keras import layers
import math
import numpy as np
import pandas as pd
import csv
import pprint

In [22]:
##---------Import of data----------------

#State/territory list (sorted alphabetically)
statelist= [[] for _ in range(58)]

#List of non-State and non-DC rows (start counting at 1)
staterowlist=[4,11,15,16,41,46,54]

#folderpath to 
folderpath = r"C:\Kevin\Applications\OSGC Undergraduate scholarship\COVID-19\COVID-19\csse_covid_19_data\csse_covid_19_daily_reports_us" + "\\"

with open(r"C:\Kevin\Applications\OSGC Undergraduate scholarship\Date List.csv") as file:
    reader = csv.reader(file)
    datelist = list(reader)
datelist=datelist[0]

for i in range(0, len(datelist)):
    splitdate = datelist[i].split("/")
    datelist[i] = splitdate[0] + "-" + splitdate[1] + "-20" + splitdate[2]
#print(datelist)

#could optimize by adding the 'columns' as a row, then flipping everything with numpy

#extract all the confirmed cases by date from all the data
for date in datelist:
    filepath = folderpath + date + ".csv"
    with open(filepath) as file:
        reader = csv.reader(file)
        data = list(reader)
    for i in range(58):
        statelist[i].append(data[i+1][5])
    
#Infection rates (Total cases) now stored in an array, with each row representing a state or region,
#and each column representing a date for the infection
    
    
#--------------Normalization of data------------   

#Make numpy recognize that data is numerical
statelist = np.array(statelist).astype(float)

#Normalize by the highest infection rate
#should maybe normalize by total population
maxstatelist = np.amax(statelist, axis=1)
statelist = statelist / maxstatelist[:,None]

#Infection rates now in terms of (cases/max one day infection cases)


#---------------Formatting of data---------------

#number of days in a grouping
time_step = 14

#grouping of data
print(statelist)
print()
print(statelist[:][:][:14])

#Infection rates are now in sets of 14 days, with the last three months of data reserved for testing

[[3.82837360e-03 4.04030702e-03 4.21883222e-03 ... 9.73275509e-01
  9.86457141e-01 1.00000000e+00]
 [1.64496562e-03 1.67520396e-03 1.72358530e-03 ... 9.77774821e-01
  9.77774821e-01 1.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.82326369e-03
  2.82326369e-03 2.82326369e-03]
 ...
 [0.00000000e+00 1.73733537e-03 1.81979482e-03 ... 9.74178818e-01
  9.88245263e-01 1.00000000e+00]
 [9.27435741e-06 2.89022702e-03 2.99730369e-03 ... 9.74692808e-01
  9.86879314e-01 1.00000000e+00]
 [0.00000000e+00 2.29440083e-03 2.35280376e-03 ... 9.85324178e-01
  9.93625737e-01 1.00000000e+00]]

[[3.82837360e-03 4.04030702e-03 4.21883222e-03 ... 9.73275509e-01
  9.86457141e-01 1.00000000e+00]
 [1.64496562e-03 1.67520396e-03 1.72358530e-03 ... 9.77774821e-01
  9.77774821e-01 1.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.82326369e-03
  2.82326369e-03 2.82326369e-03]
 ...
 [2.70314333e-03 4.56291115e-03 4.69511595e-03 ... 9.46529559e-01
  9.73903552e-01 1.00000000e+00

In [ ]:
#test


In [11]:
#hyperparameters
#input_shape=(784,)
#neuron



model = tf.keras.models.Sequential()
model.add(layers.Dense(units = 256, activation = 'tanh', input_shape = (784,)))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(units = 96, activation = 'tanh'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(units = 48, activation = 'tanh'))
model.add(layers.Dense(units = 28, activation = 'tanh'))
model.add(layers.Dense(units = 48, activation = 'tanh'))
model.add(layers.Dense(units = 12, activation = 'tanh'))
model.add(layers.Dense(units = 6, activation = 'tanh'))
model.add(tf.keras.layers.Dense(units = 10, activation = 'softmax'))
#model.add(layers.Dense(units = 1, activation = 'relu'))
#need to find a better loss and metric for compiler for covid data- paper doesn't specify much past what is here
model.compile(optimizer = 'RMSprop', loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])




In [12]:

model.fit(X_train, y_train, epochs =5)
test_loss, test_accuracy = model.evaluate(X_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.8129 - sparse_categorical_accuracy: 0.7703
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4770 - sparse_categorical_accuracy: 0.8461A: 2s - l
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4134 - sparse_categorical_accuracy: 0.8605
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3821 - sparse_categorical_accuracy: 0.8705
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.4268 - sparse_categorical_accuracy: 0.8605
